In [1]:
import pandas as pd

# Load the Excel file
file_path = 'FMI Data V2.xlsx'
xls = pd.ExcelFile(file_path)

# Define a function to transform a single sheet
def transform_sheet(df, sheet_name):
    # 1. Drop the first 9 rows
    df = df.drop(index=range(8))
    
    # Make the first row as column names
    df.columns = df.iloc[0]
    
    # 2. Drop the first row from the dataset
    df = df.drop(df.index[0])

    # 2. Drop columns where the column name is NaN
    df = df.loc[:, ~df.columns.isna()]
    
    # 5. Create a new column after 'Product_Net_Sales' called 'Pre_Post' and mark them all as 'Pre'
    df.insert(6, 'Pre_Post', 'Pre')
    
    # 6. Identify post-period columns (those after 'Product_Net_Sales')
    # post_columns = [col + 'V2' if 7 <= idx < 13 else col for idx, col in enumerate(df.columns)]
    post_columns = df.columns[7:]

    # 7. Create a DataFrame for the post-period data
    post_df = df.iloc[:, 7:].copy()
    post_df['Pre_Post'] = 'Post'

    # 8. Drop post-period columns from the original DataFrame
    df = df.drop(columns=post_columns)

    # 9. Combine the pre and post DataFrames
    df_combined = pd.concat([df, post_df], ignore_index=True)

    # 10. Add the store location column
    df_combined['store_location'] = sheet_name

    return df_combined

# Process each sheet in the Excel file and combine into a single DataFrame
all_stores_df = pd.DataFrame()
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    transformed_df = transform_sheet(df, sheet_name)
    all_stores_df = pd.concat([all_stores_df, transformed_df], ignore_index=True)

# Save the combined DataFrame to a new Excel file
output_file_path = 'FMI_Data_Test_Transformed_Combined.xlsx'
all_stores_df.to_excel(output_file_path, index=False)


Index([        'Menu Item',              'Size',              'Base',
               'Menu Type',     'Product Units', 'Product Net Sales',
                       nan,                 nan,                 nan,
               'Menu Item',              'Size',              'Base',
               'Menu Type',     'Product Units', 'Product Net Sales'],
      dtype='object', name=8)
Index(['Menu Item', 'Size', 'Base', 'Menu Type', 'Product Units',
       'Product Net Sales', 'Menu Item', 'Size', 'Base', 'Menu Type',
       'Product Units', 'Product Net Sales'],
      dtype='object', name=8)
Index(['Menu Item', 'Size', 'Base', 'Menu Type', 'Product Units',
       'Product Net Sales'],
      dtype='object', name=8)
8                  Menu Item                   Size         Base  \
9    AGG-Chicken Strip MyBox  Chicken Strips My Box          NaN   
10                       NaN                    NaN          NaN   
11                       NaN                    NaN          NaN   
12        

IndexError: index 6 is out of bounds for axis 0 with size 4

In [3]:
all_stores_df.tail()

8,Pre_Post,Menu Item,Size,Base,Menu Type,Product Units,Product Net Sales,store_location
10347,Post,NaN,NaN,NaN,Medium Buffalo,0.000319,0.000404,R298-04
10348,Post,NaN,NaN,NaN,Sweet n'Spicy,0.000319,0.000404,R298-04
10349,Post,NaN,AGG Meal for 2,NaN,Honey BBQ,0.000319,0.000857,R298-04
10350,Post,NaN,NaN,NaN,Medium Buffalo,0.000319,0.000899,R298-04
10351,Post,Grand Total,NaN,NaN,NaN,1,1,R298-04
